In [1]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import datetime

# --------------------- Chargement ---------------------
print(f"[{datetime.datetime.now()}] Chargement des fichiers...", flush=True)
Y_true = np.load('Y_test.npy')
Y_pred = np.load('Y_prediction.npy')

print(f"[{datetime.datetime.now()}] Forme des données : Y_test = {Y_true.shape}, Y_pred = {Y_pred.shape}", flush=True)

# --------------------- Évaluation ---------------------
print(f"[{datetime.datetime.now()}] Calcul des métriques...", flush=True)

# Accuracy globale
acc = accuracy_score(Y_true, Y_pred)
f1_macro = f1_score(Y_true, Y_pred, average='macro')
f1_weighted = f1_score(Y_true, Y_pred, average='weighted')

print("\n=== Résultats globaux ===")
print(f"Accuracy : {acc:.4f}")
print(f"F1-score (macro) : {f1_macro:.4f}")
print(f"F1-score (weighted) : {f1_weighted:.4f}")

# Rapport par classe
print("\n=== Rapport par classe ===")
print(classification_report(Y_true, Y_pred, digits=4))

# Matrice de confusion
print("\n=== Matrice de confusion ===")
conf_matrix = confusion_matrix(Y_true, Y_pred)
print(conf_matrix)


[2025-05-28 21:44:27.551275] Chargement des fichiers...
[2025-05-28 21:44:27.560841] Forme des données : Y_test = (848363,), Y_pred = (848363,)
[2025-05-28 21:44:27.561514] Calcul des métriques...

=== Résultats globaux ===
Accuracy : 0.9982
F1-score (macro) : 0.7772
F1-score (weighted) : 0.9981

=== Rapport par classe ===
              precision    recall  f1-score   support

           0     0.9984    0.9999    0.9991    681396
           1     0.8959    0.9387    0.9168       587
           2     0.9998    0.9995    0.9996     38408
           3     0.9931    0.9802    0.9866      3088
           4     0.9988    0.9997    0.9993     69037
           5     0.9903    0.9915    0.9909      1650
           6     0.9930    0.9845    0.9887      1739
           7     0.9996    0.9979    0.9987      2380
           8     1.0000    0.3333    0.5000         3
           9     1.0000    0.2727    0.4286        11
          10     0.9994    0.9778    0.9885     47641
          11     0.9977   

/home/zamaien/Vrac Model_IDS/papier_charles/env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/zamaien/Vrac Model_IDS/papier_charles/env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/zamaien/Vrac Model_IDS/papier_charles/env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

In [ ]:
# Étape 1 : mapping initial des classes (pour référence humaine)
original_labels = {
    0: "BENIGN",
    1: "Bot",
    2: "DDoS",
    3: "DoS GoldenEye",
    4: "DoS Hulk",
    5: "DoS Slowhttptest",
    6: "DoS slowloris",
    7: "FTP-Patator",
    8: "Heartbleed",
    9: "Infiltration",
    10: "PortScan",
    11: "SSH-Patator",
    12: "Web Attack – Brute Force",
    13: "Web Attack – Sql Injection",
    14: "Web Attack – XSS"
}

# Étape 2 : fonction de regroupement
def regroup_classes(label_id):
    if label_id in [3, 4, 5, 6]:          # Tous les DoS
        return "DoS"
    elif label_id in [12, 13, 14]:        # Web Attacks
        return "WebAttack"
    elif label_id in [7, 11]:             # Patator
        return "Patator"
    else:
        return original_labels[label_id]  # inchangé

# Étape 3 : générer l'encodage compact
intermediate_labels = [regroup_classes(i) for i in original_labels]
unique_labels = sorted(set(intermediate_labels))
label_to_compact_id = {name: idx for idx, name in enumerate(unique_labels)}
final_mapping = {old_id: label_to_compact_id[regroup_classes(old_id)] for old_id in original_labels}

# Étape 4 : appliquer à Y_test et Y_prediction
Y_test_transformed = [final_mapping[y] for y in np.load("Y_test.npy")]
Y_pred_transformed = [final_mapping[y] for y in np.load("Y_prediction.npy")]
